## Imports

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Basic
from IPython.display import display

# For OS-agnostic paths
from pathlib import Path

# Plotting
from matplotlib import pyplot as plt
import seaborn as sns
import torch
from torch import nn
import pandas as pd
import numpy as np
sns.set_style("whitegrid")
from copy import deepcopy
import glob, json
from datetime import datetime
import torch
from torch import nn

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, Dataset, TensorDataset


from torchinfo import summary

import mlflow

%cd ..

from src.utils import load_raw_data
from src.plotting import plot_pointcloud, plot_sample_figures
from src.models.HardSphereGAN import GAN
from src.models.StaticScaler import StaticMinMaxScaler
from src.HSDataset import HSDataset

%cd -

plt.set_cmap("viridis")

# Hard spheres model development


First stage: Develop a CNN - based GAN to work with ordered point clouds.'

This notebook is an attempt at the simpler hexagonal and square lattices after slow progress in the full-scale experiment.

In [ ]:

phis = [0.84] # Add more phis here

path = Path("../data/raw/samples")
path = Path("../data/raw/crystal/Hex")
path = Path("../data/raw/crystal/Sq")


files, dataframe, metadata = load_raw_data(path, phi=phis, subpath="disorder-0.2")
# files, dataframe, metadata = load_raw_data(path, phi=phis)

dataframe.sort_index()

In [ ]:
# Hex lattice

N = 1600 
X_box = 41.076212368516387
Y_box = 35.573043402379753   
 
# Square lattice
N = 1600
X_box = 38.225722823651111
y_box = 38.225722823651111 
dataframe["r"] = 0.375 # Fixed radius for all data, for square lattice

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StaticMinMaxScaler(
    columns = ["x", "y", "r"],
    maximum = [X_box, Y_box, 2*X_box], # NOTE: Tuned for physical feasibility
    minimum = [-X_box, -Y_box, 0] # NOTE: Tuned for physical feasibility
)

dataframe_scaled = pd.DataFrame(scaler.transform(dataframe), columns=dataframe.columns)

dataframe_scaled.set_index(dataframe.index, inplace=True)

dataframe_scaled = dataframe_scaled.drop(columns=["class"]) # Redundant with r
# dataframe_scaled = dataframe_scaled.sort_values(by=["experiment", "sample"])
dataframe_scaled.describe().round(7)

## Example

In [ ]:
sample = dataframe_scaled.copy().query("(sample=='sample-1')").loc[:,["x", "y", "r"]].reset_index(drop=True)
sample = torch.tensor(sample.values).unsqueeze(0)
print(sample.shape)
plot_pointcloud(sample[0], plot_radius=False)

## Train the model

In [ ]:
run_params = {
    "training":{
        "device": "mps" if torch.backends.mps.is_available() else "cpu", # MPS is not supported by PyTorch 2D TransposeConv
        "batch_size": 32,
        "epochs": 5000,
        "early_stopping_patience": 50,
        "early_stopping_headstart": 0,
        "early_stopping_tolerance": 1e-3, # Gradient norm based
        "log_image_frequency": 5,
        "generator_headstart": 0,
        "training_ratio_dg": 3,
        "optimizer_g": {
            "name": "Adam",
            "lr": 0.00005, # 0.00005, #0.002, 
            # "hypergrad_lr": 1e-6,
            "weight_decay": 0,
            "betas": (0.5, 0.999)
        },
        "optimizer_d": {
            "name": "Adam",
            "lr": 0.00005, #0.002, 
            # "hypergrad_lr": 1e-6,
            "weight_decay": 0,
            "betas": (0.5, 0.999)
        },
        "d_loss":{
            "name": "CryinGANDiscriminatorLoss", # CryinGANDiscriminatorLoss for WaGAN + L1 loss, BCELoss for baseline
            "mu": 1.0, # L1 loss coefficient
        },
        "g_loss":{
            "name": "HSGeneratorLoss",
            "radius_loss": 0,
            "grid_density_loss": 0,
            "gan_loss": 1,
            "distance_loss": 0,
            "physical_feasibility_loss": 0,
            "coefficients":{
                "gan_loss": 1,
                "radius_loss": 0,
                "grid_density_loss": 0,
                "physical_feasibility_loss": 0,
                "distance_loss": 0,
            },
        }
    },
    "dataset":{
        "descriptor_list": ["phi"],
        "synthetic_samples":{
            "rotational": 0,
            "shuffling": 0,
            "spatial_offset_static": 0.1
            }, # NOTE: Could do subsquares and more rotations.
        # "downsample": 0 #TODO: Remove
        "downsample": 0.05 #TODO: Remove
    }
}

dataset = HSDataset(
    dataframe_scaled.copy(), # Dont use the ordering
    **run_params["dataset"]
    )
dataset.y.shape

# Create model

Create a GAN architecture, which creates point clouds $\hat{y}$ based on the descriptor(s) $\hat{X}$ and a random noise vector $\hat{z}$.

In [ ]:
from src.models.CryinGAN import CCCGDiscriminator, CCCGeneratorWithDiffusion, CCCGenerator

sample_x = dataset[0:32][0].cpu()
sample_y = dataset[0:32][1].cpu()

sample_x_mps = sample_x.to("mps")
sample_y_mps = sample_y.to("mps")

print(sample_x.shape, sample_y.shape)


test_frac = 0.2

dataset = dataset.to(run_params["training"]["device"])
trainset, testset = torch.utils.data.random_split(dataset, [1-test_frac, test_frac])
print(len(trainset), len(testset))

out_samples = dataset.samples[0].shape[1]
out_dimensions = 2 #dataset.samples[0].shape[2]

kernel_size = (1,1)
stride=1

generator = CCCGenerator(
    kernel_size=kernel_size,
    stride=stride,
    channels_coefficient=1,
    rand_features=513,# 513 for one paper, 64 for another,
    out_dimensions=out_dimensions,
    out_samples=out_samples,
    latent_dim=256, # 128 for the papers
    fix_r=0.0049,
    clip_output = False
    # (
    #     dataset.y.min(dim=0).values.min(dim=0).values,
    #     dataset.y.max(dim=0).values.max(dim=0).values
    # )
    ).to("mps")


input_channels = 3

discriminator = CCCGDiscriminator(
    input_channels=input_channels, 
    in_samples=out_samples, 
    kernel_size=(1,1),
    channels_coefficient=3
    ).to("mps")

gan = GAN(
    dataset, 
    dataset,# No separate test set
    generator_model=generator,
    discriminator_model=discriminator,
    **run_params
    )

print(summary(gan.generator, input_data=sample_x, depth=2))
print(summary(gan.discriminator, input_data=sample_y_mps, depth=2))

_out = gan.generate(sample_x)[0]
# _out = _out.numpy()

plot_pointcloud(_out, plot_radius=False)
10_603_201

In [ ]:
gan.train_n_epochs(
    epochs=run_params["training"]["epochs"],
    batch_size=run_params["training"]["batch_size"],
    experiment_name="Square lattice, small sample",
)

generated = gan.generate(sample_x).cpu()

plt.scatter(generated[0][:,0], generated[0][:,1].cpu())
plt.title("Generated sample")

## Test the discriminator with random data

In [ ]:
# Test the discriminator with random data

# Generate random data
random_data = torch.rand_like(sample_y).to("mps")
random_data = torch.randn_like(sample_y).to("mps")
print(random_data.shape)

plot_pointcloud(random_data[0].cpu().numpy(), plot_radius=False)

# Test the discriminator

output = gan.discriminator(random_data)
print(output.shape)
print("Mean of discriminator output:", output.mean().item())
plt.title(f"Discriminator output: {output[0].item()}")
plt.show()